# Task 4 - Fire Evacuation Planning for the MI Building

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.vae import VariationalAutoEncoder
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from helpers.data import get_fire_evac_dataset
from helpers.plots import plot_2d_train_test
from helpers.plots import plot_2d_fire_evac_set

### 1. Loading the dataset and visualizing it

In [ ]:
train, test = get_fire_evac_dataset()
plot_2d_train_test(
    train, test, xlabel="x", ylabel="y", title="FireEvac Dataset", save_path="plots/fire_evac_data.pdf", alpha=0.5
)

### 2. Training a VAE

In [ ]:
# Data & model configuration
latent_dim = 2
batch_size = 32
hidden_size = 64
no_epochs = 200
verbosity = 1
num_channels = 1
input_shape = train[0].shape

Compared to Task 3, we reduce the batch_size because the train set is smaller

In [ ]:
# preprocess data to be in range [0,1]
def normalise(data: np.ndarray) -> np.ndarray:
    """
    Normalises the data such that all data points are in the range [0,1]
    Args:
        data (ndarray): data to be normalised

    Returns: normalised data

    """
    max = data.max()
    min = data.min()

    data_normalised = (data - min) / (max - min)
    return data_normalised

def renormalise(data_normalised: np.ndarray, data: np.ndarray) -> np.ndarray:
    """
    processes normalised data such that the values are similar to the original data
    Args:
        data_normalised (ndarray): normalised data in range [0,1]
        data (ndarray): original data

    Returns: data that has similar values than the original data

    """
    max = data.max()
    min = data.min()
    data_renormalised = data_normalised * (max - min) + min
    return data_renormalised

train_normalised = normalise(train)
test_normalised = normalise(test)

Even though it is suggested to normalise the data to the interval [-1,1], we receive better results for interval [0,1]

In [ ]:
vae = VariationalAutoEncoder(input_shape=input_shape, latent_dim=latent_dim, hidden_size=hidden_size)

Hyperparameters were tuned and the network was tested for:
- different dimensions of the latent space (2-32): latent_dim=2 returned best results
- different batch sizes (8-128), starting with a smaller batch size compared to Task 3 because the train set is smaller: batch_size=32 returned best results
- different hidden size (64-1024): hidden_size=64 returned best results
- different number of Dense layers (2-4): best results for 2 Dense layers
- different learning_rates (0.0001-0.001): best results for learning_rate=0.0005

In [ ]:
# Compile VAE
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", patience=50)
opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=opt)
# Train autoencoder
history = vae.fit(train_normalised, train_normalised, epochs=no_epochs, batch_size=batch_size, validation_data=(test_normalised, test_normalised), callbacks=[early_stopping])

In [ ]:
# print loss history
loss_history = history.history['val_loss']

plt.plot(loss_history)
plt.title("loss curve of ELBO for test set")
plt.xlabel('#iterations')
plt.ylabel('-ELBO loss')
plt.show()
plt.savefig("plots/fire_evac_loss.pdf", bbox_inches="tight")

### 3. Plot of the reconstructed test set

In [ ]:
plot_2d_fire_evac_set(test, xlabel="x", ylabel="y", title="FireEvac Test Set", save_path="plots/fire_evac_test_data.pdf", alpha=0.5)

test_reconstructed = np.empty_like(test_normalised)
for i in range(test_normalised.shape[0]):
    _, _, z = vae.encoder(test_normalised[i-1].reshape(-1, 2))
    test_sample = vae.decoder(z)[0]
    test_reconstructed[i-1] = test_sample

test_reconstructed = renormalise(test_reconstructed, test)
plot_2d_fire_evac_set(test_reconstructed, xlabel="x", ylabel="y", title="FireEvac Reconstructed Test Set", save_path="plots/fire_evac_test_data_reconstruction.pdf", alpha=0.5)

### 4. Plot of generated samples

In [ ]:
# TODO
# randomly choose input for decoder
# decode
# plot generated samples with plot fire evac set

def generate_samples(num_samples: int) -> np.ndarray:
    """
    Visualises generation of num_samples digits

    Args:
        num_samples (int): number of samples to be generated

    Returns: array of sampled data
    """
    generated_samples = np.empty(shape=(num_samples, 2))
    for i in range(num_samples):
        z_sample = np.random.normal(size=latent_dim) 
        out = vae.decoder(np.array([z_sample]))[0]
        generated_samples[i] = out
    return renormalise(generated_samples, train)

generated_samples = generate_samples(1000)

plot_2d_fire_evac_set(generated_samples, xlabel="x", ylabel="y", title="FireEvac Generated Set", save_path="plots/fire_evac_test_data_generation.pdf", alpha=0.5)

### 5. Generate data to estimate the critical number of people for the MI building

In [ ]:
def count_sensitive_area(samples: np.ndarray) -> int:
    """
    counts the number of samples inside the sensitive area [130<x<150, 50<y<70]
    Args:
        samples (ndarray): array of samples with positions

    Returns: number of samples inside the sensitive area

    """
    count = 0
    for sample in samples:
        x, y = sample
        if x > 130 and x < 150 and y > 50 and y < 70:
            count += 1
    return count

for i in range(950,1100):
    generated_samples = generate_samples(i)
    count = count_sensitive_area(generated_samples)
    if count > 95:
        print("For " + str(i) + " samples, there are " + str(count) + " people in the sensitive area")
    if count > 100:
        print("CRITICAL NUMBER IS REACHED")

The critical number at the main entrance is reached for approximately 1000 samples (then there are more than 100 people in the area [130,150][50,70]). Approximately 10% of all people are located in the sensitive area. If this area should not allow more than 100 people for safety reasons, then a smaller number needs to be chosen because in some cases, the number is exceeded for 900-950 people in the MI building.

### Bonus:

In [ ]:
# generation of 100 people in the MI building
generated_positions = generate_samples(100)
generated_positions = renormalise(generated_positions, train)

plot_2d_fire_evac_set(generated_positions, xlabel="x", ylabel="y", title="FireEvac Reconstructed Test Set", save_path="plots/fire_evac_test_data_generation_bonus.pdf", alpha=0.5)